In [4]:
!pip install pandas scikit-learn



In [ ]:
The data parser file

In [24]:
# /usr/bin/env python3
import itertools
import pandas
from sklearn.metrics.pairwise import haversine_distances
from math import radians


def distance(row1, row2):
    """Computes the Haversine distance between two towns in the database.
    Source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html"""
    pos1 = (row1['coordinates']['lat'], row1['coordinates']['lon'])
    pos2 = (row2['coordinates']['lat'], row2['coordinates']['lon'])
    radians1 = [radians(pos1[0]), radians(pos1[1])]
    radians2 = [radians(pos2[0]), radians(pos2[1])]
    res = haversine_distances([radians1, radians2])
    res *= 6371000 / 1000  # multiply by Earth radius to get kilometers
    return res[0][1]


# The following is a simple example of parsing the dataset. You are free to extract any subset of the cities
# to benchmark your LP/ILP solutions. You can also transform the dataset into some intermediary data that is
# easier to parse -- for example, creating the distance matrix explicitly, and not just implicitly.

# You are also free to implement the solution in a language other than Python. In that case, please use the same
# dataset, so that the results are somewhat comparable.

# Data source: https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/information/?disjunctive.cou_name_en&sort=name

df = pandas.read_json("dataset.json")
print(f"There are {len(df.index)} entries in the file overall.")
europe = df[df['timezone'].str.contains("Europe")]
print(f"There are {len(europe.index)} entries for towns in Europe.")
first_in_europe = europe.iloc[0]
second_in_europe = europe.iloc[-1]
print(f"The distance between {first_in_europe['name']} and {second_in_europe['name']} is "
      f"{distance(first_in_europe, second_in_europe)} km.")

poland = europe[europe['country_code'].str.contains('PL')]
print(f"There are {len(poland.index)} entries for towns in Poland.")
first_in_poland = poland.iloc[0]
second_in_poland = poland.iloc[1]

print(f"The distance between {first_in_poland['name']} and {second_in_poland['name']} is "
      f"{distance(first_in_poland, second_in_poland)} km.")

# The actual distance between Żurowa and Wyśmierzyce seems to be 219 kms, and the function claims 201 km.


There are 140807 entries in the file overall.
There are 69390 entries for towns in Europe.
The distance between Cerreto Castello and Saint-Melaine-sur-Aubance is 692.2505607394098 km.
There are 2872 entries for towns in Poland.
The distance between Żurowa and Wyśmierzyce is 201.5473452215105 km.


# To find the Pair wise distance between any city We have sorted the poland's cities only

In [33]:
import itertools
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians

def distance(row1, row2):
    pos1 = (row1['coordinates']['lat'], row1['coordinates']['lon'])
    pos2 = (row2['coordinates']['lat'], row2['coordinates']['lon'])
    radians1 = [radians(pos1[0]), radians(pos1[1])]
    radians2 = [radians(pos2[0]), radians(pos2[1])]
    res = haversine_distances([radians1, radians2])
    res *= 6371000 / 1000  # multiply by Earth radius to get kilometers
    return res[0][1]

# Load the dataset
df = pd.read_json("dataset.json")

# Extract relevant columns for the first 500 cities in Poland
poland_data = df[df['country_code'] == 'PL'].head(500)[['name', 'coordinates']]

# Initialize an empty list to store pairs of cities and distances
distances = []

# Iterate through each pair of cities in the first 50 cities in Poland
for (city1_idx, city1), (city2_idx, city2) in itertools.combinations(poland_data.iterrows(), 2):
    dist = distance(city1, city2)
    
    # Append the pair and distance to the list
    distances.append((city1['name'], city2['name'], dist))

# Create a dataframe from the distances
distance_df = pd.DataFrame(distances, columns=['From', 'To', 'Distance'])

# Store the distances in an Excel file
distance_df.to_excel('poland_500_cities_pairwise_distances.xlsx', index=False)


the distance from Żurowa to all other cities in the dataset

In [21]:
# /usr/bin/env python3
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import itertools

def distance(row1, row2):
    # Computes the Haversine distance between two towns in the database
    pos1 = (row1['coordinates']['lat'], row1['coordinates']['lon'])
    pos2 = (row2['coordinates']['lat'], row2['coordinates']['lon'])
    radians1 = [radians(pos1[0]), radians(pos1[1])]
    radians2 = [radians(pos2[0]), radians(pos2[1])]
    res = haversine_distances([radians1, radians2])
    res *= 6371000 / 1000  # multiply by Earth radius to get kilometers
    return res[0][1]

# Load the dataset
df = pd.read_json("dataset.json")

# Select the city of Żurowa
zurowa = df[df['name'] == 'Żurowa']
poland = df[df['country_code'] == 'PL']

# Calculate the distance from Żurowa to all other cities in the dataset
distances = []
for index, row in poland.iterrows():
    dist = distance(zurowa.iloc[0], row)
    distances.append((zurowa['name'].values[0], row['name'], dist))

# Create a dataframe from the distances
distance_df = pd.DataFrame(distances, columns=['From', 'To', 'Distance'])

# Store the distances in an Excel file
distance_df.to_excel('zurowa_distances.xlsx', index=False)

# Create a subset of 500 cities
subset = distance_df.nsmallest(500, 'Distance')

# Store the subset in an Excel file
subset.to_excel('zurowa_distances_to_polish_cities_subset.xlsx', index=False)

If this one execute the output will show each pairwise_distances.xlsx, all of the citie in dataset.json file.

In [22]:
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians

def haversine_distance(pos1, pos2):
    radians1 = [radians(pos1[0]), radians(pos1[1])]
    radians2 = [radians(pos2[0]), radians(pos2[1])]
    res = haversine_distances([radians1, radians2])
    res *= 6371000 / 1000  # multiply by Earth radius to get kilometers
    return res[0][1]

# Load the dataset
df = pd.read_json("dataset.json")

# Extract relevant columns
city_data = df[['name', 'coordinates']]

# Initialize an empty list to store pairs of cities and distances
distances = []

# Iterate through each pair of cities
for (city1, coord1), (city2, coord2) in itertools.combinations(city_data.iterrows(), 2):
    dist = haversine_distance((coord1['lat'], coord1['lon']), (coord2['lat'], coord2['lon']))
    
    # Append the pair and distance to the list
    distances.append((city1, city2, dist))

# Create a dataframe from the distances
distance_df = pd.DataFrame(distances, columns=['From', 'To', 'Distance'])

# Store the distances in an Excel file
distance_df.to_excel('pairwise_distances.xlsx', index=False)


KeyError: 'lat'